# Notebook to start overviewing the data from ECG channel

## Import libraries

In [19]:
import os
import mne
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['figure.dpi'] = 120

%matplotlib tk

# Part 1: Data Loading

## Load ECG Signal from Parquet

In [20]:
# Constants
sampling_rate = 1000
data_folder = './rest_data_v2'
subject_id = "032"

# Load single subject data for exploration
parquet_file = f"sub-{subject_id}_task-rest_ecg_rest.parquet"
data = pd.read_parquet(os.path.join(data_folder, parquet_file))
# ...existing code...
ecg_signal = data['ecg_nk']
raw_signal = data['ecg_raw']

print(f"✅ Data loaded successfully")
print(f"   Shape: {data.shape}")
print(f"   Columns: {list(data.columns)}")

✅ Data loaded successfully
   Shape: (227081, 4)
   Columns: ['time', 'ecg_raw', 'ecg_mod', 'ecg_nk']


# Part 2: Analysis & Metrics Extraction

## Process ECG Signal

Clean the signal and extract R-peaks for wave delineation

In [21]:
# Process the ECG signal
signals, info = nk.ecg_process(ecg_signal, sampling_rate=sampling_rate)

print(f"✅ ECG processing complete")
print(f"   Signals shape: {signals.shape}")
print(f"   Available metrics: {list(info.keys())}")

✅ ECG processing complete
   Signals shape: (227081, 19)
   Available metrics: ['method_peaks', 'method_fixpeaks', 'ECG_R_Peaks', 'ECG_R_Peaks_Uncorrected', 'ECG_fixpeaks_ectopic', 'ECG_fixpeaks_missed', 'ECG_fixpeaks_extra', 'ECG_fixpeaks_longshort', 'ECG_fixpeaks_method', 'ECG_fixpeaks_rr', 'ECG_fixpeaks_drrs', 'ECG_fixpeaks_mrrs', 'ECG_fixpeaks_s12', 'ECG_fixpeaks_s22', 'ECG_fixpeaks_c1', 'ECG_fixpeaks_c2', 'sampling_rate', 'ECG_P_Peaks', 'ECG_P_Onsets', 'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets', 'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets']


In [22]:
# Extract R-peaks and clean signal for delineation
rpeaks = info['ECG_R_Peaks']
clean = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)

print(f"✅ R-peaks extracted and signal cleaned")
print(f"   Number of R-peaks: {len(rpeaks)}")
print(f"   Clean signal shape: {clean.shape}")

✅ R-peaks extracted and signal cleaned
   Number of R-peaks: 306
   Clean signal shape: (227081,)


In [23]:
# Perform wave delineation to identify P, QRS, and T waves
signals_delineate, waves = nk.ecg_delineate(clean, rpeaks=rpeaks, sampling_rate=sampling_rate, method="cwt", show=True)

print(f"✅ Wave delineation complete")
print(f"   P-waves: {len(waves['ECG_P_Peaks'])}")
print(f"   Q-waves: {len(waves['ECG_Q_Peaks'])}")
print(f"   S-waves: {len(waves['ECG_S_Peaks'])}")
print(f"   T-waves: {len(waves['ECG_T_Peaks'])}")

✅ Wave delineation complete
   P-waves: 305
   Q-waves: 306
   S-waves: 306
   T-waves: 305


## Calculate Heart Rate Variability (HRV)

Compute time, frequency, and non-linear domain HRV indices

In [24]:
# Calculate HRV metrics (Time, Frequency, and Non-Linear domains)
hrv_indices = nk.hrv(info, sampling_rate=sampling_rate, show=True)

print(f"✅ HRV analysis complete")
print(f"   Computed {len(hrv_indices.columns)} HRV indices")
display(hrv_indices)

✅ HRV analysis complete
   Computed 91 HRV indices


,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,741.570492,68.291334,18.416614,66.38771,NaN,NaN,NaN,NaN,50.991195,51.073157,...,1.297026,7.179886,1.051604,1.384076,1.319111,1.929557,1.660776,1.83239,2.412031,0.78468


# Part 3: Visualization & Results

## ECG Signal with Delineated Waves

Visualize processed ECG with P, QRS, and T waves identified

In [25]:
nk.ecg_plot(signals, info)


In [26]:
nk.ecg_peaks(clean, sampling_rate = 1000, show = True)

(        ECG_R_Peaks
 0                 0
 1                 0
 2                 0
 3                 0
 4                 0
 ...             ...
 227076            0
 227077            0
 227078            0
 227079            0
 227080            0
 
 [227081 rows x 1 columns],
 {'method_peaks': 'neurokit',
  'method_fixpeaks': 'None',
  'ECG_R_Peaks': array([   548,   1140,   1738,   2337,   2957,   3619,   4335,   5073,
           5821,   6510,   7169,   7811,   8440,   9082,   9791,  10745,
          11600,  12336,  13014,  13689,  14365,  15061,  15827,  16636,
          17387,  18118,  18904,  19712,  20520,  21263,  21957,  22639,
          23356,  24097,  24860,  25614,  26327,  27046,  27824,  28615,
          29388,  30114,  30836,  31600,  32449,  33259,  33989,  34711,
          35471,  36252,  37057,  37795,  38531,  39307,  40102,  40863,
          41607,  42580,  43525,  44395,  45142,  45838,  46551,  47284,
          48093,  48958,  49746,  50570,  51402,  52199,  52

## Segment

In [27]:
cqrs_epochs = nk.ecg_segment(clean, rpeaks=rpeaks, sampling_rate=1000, show=True)

## Summary Statistics

Key metrics and diagnostic information

In [28]:
# Display comprehensive summary of analysis
print("\n" + "="*70)
print("📊 ECG ANALYSIS SUMMARY")
print("="*70)

print(f"\n📈 Recording Details:")
print(f"   Duration: {len(ecg_signal)/sampling_rate:.2f} seconds")
print(f"   Sampling Rate: {sampling_rate} Hz")
print(f"   Total Samples: {len(ecg_signal):,}")

print(f"\n❤️  Heart Rate Analysis:")
print(f"   R-peaks Detected: {len(info['ECG_R_Peaks'])}")
print(f"   Average HR: {signals['ECG_Rate'].mean():.1f} bpm")
print(f"   HR Min: {signals['ECG_Rate'].min():.1f} bpm")
print(f"   HR Max: {signals['ECG_Rate'].max():.1f} bpm")

print(f"\n🌊 Wave Delineation:")
print(f"   P-waves: {len(info['ECG_P_Peaks'])} detected")
print(f"   Q-waves: {len(info['ECG_Q_Peaks'])} detected")
print(f"   S-waves: {len(info['ECG_S_Peaks'])} detected")
print(f"   T-waves: {len(info['ECG_T_Peaks'])} detected")

print(f"\n📋 HRV Indices Calculated:")
print(f"   Total: {len(hrv_indices.columns)} indices")
print(f"\n   Top HRV Metrics:")
for col in list(hrv_indices.columns)[:5]:
    print(f"   - {col}: {hrv_indices[col].values[0]:.4f}")

print("\n" + "="*70)


📊 ECG ANALYSIS SUMMARY

📈 Recording Details:
   Duration: 227.08 seconds
   Sampling Rate: 1000 Hz
   Total Samples: 227,081

❤️  Heart Rate Analysis:
   R-peaks Detected: 306
   Average HR: 81.0 bpm
   HR Min: 61.7 bpm
   HR Max: 101.4 bpm

🌊 Wave Delineation:
   P-waves: 306 detected
   Q-waves: 306 detected
   S-waves: 306 detected
   T-waves: 306 detected

📋 HRV Indices Calculated:
   Total: 91 indices

   Top HRV Metrics:
   - HRV_MeanNN: 741.5705
   - HRV_SDNN: 68.2913
   - HRV_SDANN1: 18.4166
   - HRV_SDNNI1: 66.3877
   - HRV_SDANN2: nan

